In [4]:
pip install pulp cplex scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 16.9 MB/s eta 0:00:00


The general form of LP problems we will consider is:

        min_x  cᵀx
   subject to  Gx ≥ h
               Ax = b
               l ≤ x ≤ u

The benchmark datasets use the .mps format for LP problems so there is a function to extract the parameters from this file type into the form above. But since I haven't figured out how to access these datasets yet there is also a function to generate large feasible LP example problems in the .mps format. Finally, there is a preliminary function that implements the PDHG algorithm but not yet any of the enhancments.

In [5]:
import numpy as np
from scipy.sparse import random as sparse_random
import pulp

# This function generates large feasible LP problems to test and saves them in the .mps format
def generate_feasible_lp(num_vars=100, num_ineq=200, num_eq=50, density=0.05, mps_filename="generated_lp.mps"):

    # The number of variables in each constraint with nonzero coefficients will be roughly density * num_vars

    rng = np.random.default_rng(0)

    # Step 1: Feasible solution
    x_feas = rng.uniform(low=-10, high=10, size=(num_vars, 1))

    # Step 2: Sparse matrices (convert to dense)
    G_sparse = sparse_random(num_ineq, num_vars, density=density, format='csr', random_state=None)
    A_sparse = sparse_random(num_eq, num_vars, density=density, format='csr', random_state=None)

    G = G_sparse.toarray()
    A = A_sparse.toarray()

    # Step 3: RHS vectors
    h = G @ x_feas + rng.uniform(0.1, 5.0, size=(num_ineq, 1))
    b = A @ x_feas

    # Step 4: Bounds
    l = x_feas - rng.uniform(1, 5, size=(num_vars, 1))
    u = x_feas + rng.uniform(1, 5, size=(num_vars, 1))
    l = np.maximum(l, -1e4)
    u = np.minimum(u, 1e4)

    # Step 5: Objective
    c = rng.normal(size=(num_vars, 1))

    # Step 6: Write to MPS using pulp
    prob = pulp.LpProblem("Feasible_LP", pulp.LpMinimize)
    x_vars = [
        pulp.LpVariable(f"x{i}", lowBound=float(l[i]), upBound=float(u[i]))
        for i in range(num_vars)
    ]
    prob += pulp.lpDot(c.flatten(), x_vars)

    # Inequality constraints: Gx <= h
    for i in range(num_ineq):
        prob += pulp.lpDot(G[i], x_vars) <= float(h[i]), f"ineq_{i}"

    # Equality constraints: Ax = b
    for i in range(num_eq):
        prob += pulp.lpDot(A[i], x_vars) == float(b[i]), f"eq_{i}"

    prob.writeMPS(mps_filename)
    print(f"✅ LP written to: {mps_filename}")

In [6]:
import numpy as np
import cplex
from cplex.exceptions import CplexError

# This function extracts the parameters of the LP from the .mps format to the general form
def mps_to_standard_form(mps_file):
    try:
        cpx = cplex.Cplex(mps_file)
        cpx.set_results_stream(None)  # mute output

        # Number of variables and constraints
        num_vars = cpx.variables.get_num()
        num_constraints = cpx.linear_constraints.get_num()

        # Objective vector (c)
        c = np.array(cpx.objective.get_linear())

        # Constraint matrix
        A_full = cpx.linear_constraints.get_rows()
        senses = cpx.linear_constraints.get_senses()
        rhs = np.array(cpx.linear_constraints.get_rhs())

        A = []
        G = []
        b = []
        h = []

        for i, (row, sense, rhs_i) in enumerate(zip(A_full, senses, rhs)):
            row_vec = np.zeros(num_vars)
            for idx, val in zip(row.ind, row.val):
                row_vec[idx] = val
            if sense == 'E':  # Equality constraint
                A.append(row_vec)
                b.append(rhs_i)
            elif sense == 'G':  # Greater than or equal
                G.append(row_vec)
                h.append(rhs_i)
            elif sense == 'L':  # Less than or equal
                # convert to -Gx ≥ -h
                G.append(-row_vec)
                h.append(-rhs_i)

        A = np.array(A) if A else np.zeros((0, num_vars))
        b = np.array(b) if b else np.zeros(0)
        G = np.array(G) if G else np.zeros((0, num_vars))
        h = np.array(h) if h else np.zeros(0)

        # Bounds
        l = np.array(cpx.variables.get_lower_bounds())
        u = np.array(cpx.variables.get_upper_bounds())

        c = c.reshape(-1, 1)
        h = h.reshape(-1, 1)
        b = b.reshape(-1, 1)
        l = l.reshape(-1, 1)
        u = u.reshape(-1, 1)

        return c, G, h, A, b, l, u

    except CplexError as e:
        print("CPLEX Error:", e)
        return None

In [7]:
import numpy as np

def proj_Lam(lam):
    return lam

def pdhg(c, G, h, A, b, l, u, max_iter=1000, tol=1e-4):
    """
    Solves:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u
    using PDHG algorithm.
    """

    # Define Parameters
    K = np.vstack([G, A])
    q = np.vstack([h, b])

    eta = 0.9/np.linalg.norm(K, 2)
    omega = 10

    tau = eta/omega
    sigma = eta*omega

    m_1 = G.shape[0]
    m_2 = A.shape[0]
    n = c.shape[0]

    # Termination Parameters
    tol_prim = tol * (1 + np.linalg.norm(c))
    tol_dual = tol * (1 + np.linalg.norm(q))

    # Initialize Primal and Dual Variables
    x = np.minimum(np.maximum(np.zeros((n, 1)), l), u)
    y = np.zeros((m_1 + m_2, 1))

    for i in range(max_iter):

        # Primal update
        x_old = x.copy()
        # Project x onto box constraints l ≤ x ≤ u
        grad = c - K.T @ y
        x = np.minimum(np.maximum(x - tau * grad, l), u)

        # Dual update
        y += sigma * (q - K @ (2*x - x_old))
        y[:m_1] = np.maximum(y[:m_1], 0)  # project onto constraint y:m_1 ≥ 0

        # Check Termination Criteria Periodically
        if i%1000 == 0:

            dual_op = (q.T @ y)[0][0]
            prim_op = (c.T @ x)[0][0]

            lam_p_op = (l.T @ np.maximum(grad, 0))[0][0]
            lam_n_op = (u.T @ np.minimum(grad, 0))[0][0]

            print(dual_op + lam_p_op + lam_n_op, prim_op, i)

            #print(np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])), np.linalg.norm(grad - proj_Lam(grad)), np.abs(dual_op + lam_p_op - lam_n_op - prim_op))
            #print(tol_dual, tol_prim, tol * (1 + np.abs(dual_op + lam_p_op - lam_n_op) + np.abs(prim_op)))
            if (
                np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)])) <= tol_dual
                and np.linalg.norm(grad - proj_Lam(grad)) <= tol_prim
                and np.abs(dual_op + lam_p_op + lam_n_op - prim_op) <= tol * (1 + np.abs(dual_op + lam_p_op + lam_n_op) + np.abs(prim_op))
            ):
                break

    # Returns minimal objective value, minimizer estimate in list format, and the number of iterations run
    return (c.T @ x)[0][0], x.T[0].tolist(), i + 1

In [46]:
import torch
import numpy as np

def ruiz_precondition_torch(K, c, b, h, l, u, max_iter=50, eps=1e-6):
    """
    Applies Ruiz scaling preconditioning to linear programming problem matrices using PyTorch.

    This method balances the constraint matrix and adjusts related vectors to improve
    numerical stability and conditioning of the problem.

    Args:
        K: Constraint matrix (m x n) as NumPy array
        c: Objective function coefficient vector (n x 1) as NumPy array
        b: Equality constraint RHS vector (m_eq x 1) as NumPy array
        h: Inequality constraint RHS vector (m_ineq x 1) as NumPy array
        l: Lower bound vector (n x 1) as NumPy array
        u: Upper bound vector (n x 1) as NumPy array
        max_iter: Maximum number of scaling iterations (default: 50)
        eps: Tolerance for considering a norm as zero (default: 1e-6)

    Returns:
        K_scaled: Scaled constraint matrix as NumPy array
        c_tilde: Scaled objective coefficients as NumPy array
        b_tilde: Scaled equality RHS as NumPy array
        h_tilde: Scaled inequality RHS as NumPy array
        l_tilde: Scaled lower bounds as NumPy array
        u_tilde: Scaled upper bounds as NumPy array
        D1: Row scaling factors (m x 1) as NumPy array
        D2: Column scaling factors (n x 1) as NumPy array
    """
    # Convert NumPy inputs to PyTorch tensors
    K_tensor = torch.from_numpy(K).float()
    c_tensor = torch.from_numpy(c).float() if c is not None else None
    b_tensor = torch.from_numpy(b).float() if b is not None else None
    h_tensor = torch.from_numpy(h).float() if h is not None else None
    l_tensor = torch.from_numpy(l).float() if l is not None else None
    u_tensor = torch.from_numpy(u).float() if u is not None else None

    # Get problem dimensions
    m, n = K_tensor.shape
    m_ineq = h_tensor.shape[0] if h is not None else 0
    m_eq = b_tensor.shape[0] if b is not None else 0

    # Initialize scaling factors as identity
    D1 = torch.ones(m, dtype=torch.float32)  # Row scaling factors
    D2 = torch.ones(n, dtype=torch.float32)  # Column scaling factors
    K_scaled = K_tensor.clone()  # Work on a copy of the constraint matrix

    # Main Ruiz scaling loop
    for it in range(max_iter):
        # Step 1: Row scaling (balance row norms)
        # Compute row-wise infinity norms (max absolute value in each row)
        row_norms, _ = torch.max(torch.abs(K_scaled), dim=1)
        # Replace near-zero norms with 1 to avoid division issues
        row_norms[row_norms < eps] = 1.0
        # Compute scaling factors as square roots of norms
        row_scale = torch.sqrt(row_norms)
        # Update cumulative row scaling factors
        D1 /= row_scale
        # Apply row scaling to matrix
        K_scaled = K_scaled / row_scale.view(-1, 1)  # Broadcasting along rows

        # Step 2: Column scaling (balance column norms)
        # Compute column-wise infinity norms (max absolute value in each column)
        col_norms, _ = torch.max(torch.abs(K_scaled), dim=0)
        # Replace near-zero norms with 1 to avoid division issues
        col_norms[col_norms < eps] = 1.0
        # Compute scaling factors as square roots of norms
        col_scale = torch.sqrt(col_norms)
        # Update cumulative column scaling factors
        D2 /= col_scale
        # Apply column scaling to matrix
        K_scaled = K_scaled / col_scale.view(1, -1)  # Broadcasting along columns

        # Check convergence: stop if all row and column max norms are near 1
        row_max, _ = torch.max(torch.abs(K_scaled), dim=1)
        col_max, _ = torch.max(torch.abs(K_scaled), dim=0)
        if (torch.max(torch.abs(1 - row_max)) < 0.1 and
            torch.max(torch.abs(1 - col_max)) < 0.1):
            break  # Exit early if convergence criteria met

    # Convert scaling factors to column vectors for consistency
    D1 = D1.view(-1, 1)
    D2 = D2.view(-1, 1)

    # Apply scaling to objective function coefficients
    c_tilde = c_tensor * D2 if c is not None else None

    # Apply scaling to variable bounds
    l_tilde = l_tensor / D2 if l is not None else None
    u_tilde = u_tensor / D2 if u is not None else None

    # Apply scaling to constraint right-hand sides
    if h is not None:
        h_tilde = h_tensor * D1[:m_ineq]
    else:
        h_tilde = None

    if b is not None:
        b_tilde = b_tensor * D1[m_ineq:]
    else:
        b_tilde = None

    # Convert all results back to NumPy arrays
    K_scaled_np = K_scaled.numpy() if K_scaled is not None else None
    c_tilde_np = c_tilde.numpy() if c_tilde is not None else None
    b_tilde_np = b_tilde.numpy() if b_tilde is not None else None
    h_tilde_np = h_tilde.numpy() if h_tilde is not None else None
    l_tilde_np = l_tilde.numpy() if l_tilde is not None else None
    u_tilde_np = u_tilde.numpy() if u_tilde is not None else None
    D1_np = D1.numpy() if D1 is not None else None
    D2_np = D2.numpy() if D2 is not None else None

    # Return scaled problem components and scaling factors as NumPy arrays
    return (K_scaled_np, c_tilde_np, b_tilde_np, h_tilde_np,
            l_tilde_np, u_tilde_np, D1_np, D2_np)

In [47]:
import numpy as np

def proj_Lam(lam):
    """Projection function for the dual variable (identity function in this context)."""
    return lam

def pdhg_precond(c, G, h, A, b, l, u, max_iter=1000, tol=1e-6):
    """
    Solves linear programming problem with PDHG algorithm using Ruiz preconditioning:
        min cᵀx s.t. Gx ≥ h, Ax = b, l ≤ x ≤ u

    The Ruiz preconditioning improves numerical conditioning by balancing the constraint matrix.

    Args:
        c: Objective coefficient vector (n x 1)
        G: Inequality constraint matrix (m_ineq x n)
        h: Inequality constraint RHS (m_ineq x 1)
        A: Equality constraint matrix (m_eq x n)
        b: Equality constraint RHS (m_eq x 1)
        l: Lower bounds (n x 1)
        u: Upper bounds (n x 1)
        max_iter: Maximum iterations for PDHG
        tol: Tolerance for termination criteria

    Returns:
        objective_value: Optimal objective value
        solution: Minimizer estimate in list format
        iterations: Number of iterations run
    """
    # Save original parameters for solution recovery
    c_orig = c.copy()  # Preserve original objective coefficients
    G_orig = G.copy()  # Preserve original inequality matrix
    h_orig = h.copy()  # Preserve original inequality RHS
    A_orig = A.copy()  # Preserve original equality matrix
    b_orig = b.copy()  # Preserve original equality RHS
    l_orig = l.copy()  # Preserve original lower bounds
    u_orig = u.copy()  # Preserve original upper bounds

    # === Ruiz Preconditioning Section ===
    # Combine all constraints into a single matrix
    K_full = np.vstack([G, A])

    # Apply Ruiz scaling to the problem
    # This balances the constraint matrix and scales related parameters
    K_scaled, c, b, h, l, u, D1, D2 = ruiz_precondition(
        K_full, c, b, h, l, u
    )

    # Extract scaled constraint matrices
    m_ineq = G.shape[0]  # Number of inequality constraints
    G = K_scaled[:m_ineq]  # Scaled inequality constraint matrix
    A = K_scaled[m_ineq:]  # Scaled equality constraint matrix

    # === PDHG Algorithm with Scaled Parameters ===
    # Define full constraint system for PDHG
    K = K_scaled  # Combined scaled constraint matrix
    q = np.vstack([h, b])  # Combined scaled constraint RHS

    # Calculate step sizes based on scaled matrix norm
    norm_K = np.linalg.norm(K, 2)  # Spectral norm of scaled matrix
    eta = 0.9 / norm_K  # Scaling factor for step sizes
    omega = 10  # Ratio parameter for primal/dual step sizes

    tau = eta / omega  # Primal step size
    sigma = eta * omega  # Dual step size

    # Problem dimensions
    m_1 = G.shape[0]  # Number of scaled inequality constraints
    m_2 = A.shape[0]  # Number of scaled equality constraints
    n = c.shape[0]  # Number of variables

    # Adaptive termination tolerances
    tol_prim = tol * (1 + np.linalg.norm(c))  # Primal feasibility tolerance
    tol_dual = tol * (1 + np.linalg.norm(q))  # Dual feasibility tolerance

    # Initialize primal and dual variables
    x = np.minimum(np.maximum(np.zeros((n, 1)), l), u)  # Primal variable with box projection
    y = np.zeros((m_1 + m_2, 1))  # Dual variable (Lagrange multipliers)

    # Main PDHG iteration loop
    for i in range(max_iter):
        # Save previous primal iterate for extrapolation
        x_old = x.copy()

        # Primal update:
        # 1. Compute gradient: c - Kᵀy
        # 2. Take step in negative gradient direction: x - τ * gradient
        # 3. Project onto box constraints [l, u]
        grad = c - K.T @ y
        x = np.minimum(np.maximum(x - tau * grad, l), u)

        # Dual update:
        # 1. Compute constraint violation: q - K(2x - x_old)
        # 2. Take step in positive direction: y + σ * violation
        # 3. Project inequality multipliers to non-negative orthant
        y += sigma * (q - K @ (2*x - x_old))
        y[:m_1] = np.maximum(y[:m_1], 0)  # Project inequality multipliers to ≥0

        # Check termination criteria periodically
        if i % 1000 == 0:
            # Calculate dual objective value
            dual_op = (q.T @ y)[0][0]

            # Calculate primal objective value
            prim_op = (c.T @ x)[0][0]

            # Calculate complementarity terms for box constraints
            lam_p_op = (l.T @ np.maximum(grad, 0))[0][0]  # Lower bound complementarity
            lam_n_op = (u.T @ np.minimum(grad, 0))[0][0]  # Upper bound complementarity

            # Print progress
            print(f"Duality gap: {dual_op + lam_p_op + lam_n_op - prim_op:.6f}, "
                  f"Primal obj: {prim_op:.6f}, Iter: {i}")

            # Check termination criteria:
            # 1. Primal feasibility: ||Ax - b|| + ||[Gx - h]_+||
            primal_feas = np.linalg.norm(np.vstack([A @ x - b, np.maximum(h - G @ x, 0)]))

            # 2. Dual feasibility: distance to projected gradient
            dual_feas = np.linalg.norm(grad - proj_Lam(grad))

            # 3. Duality gap: |dual_obj + comp_terms - primal_obj|
            gap = np.abs(dual_op + lam_p_op + lam_n_op - prim_op)

            # Scale tolerances by problem magnitude
            scaled_tol = tol * (1 + np.abs(dual_op + lam_p_op + lam_n_op) + np.abs(prim_op))

            if (primal_feas <= tol_dual and
                dual_feas <= tol_prim and
                gap <= scaled_tol):
                print(f"Converged at iteration {i}")
                break

    # === Solution Recovery ===
    # Transform solution back to original space: x_original = D2 * x_scaled
    x_original = x * D2

    # Calculate objective value using original parameters
    objective_value = (c_orig.T @ x_original)[0][0]

    # Return results
    return objective_value, x_original.T[0].tolist(), i + 1

Testing the algorithm

In [48]:
from time import perf_counter

# Makes a timer to measure code omtimality
class Timer:
    def __enter__(self):
        self.start = perf_counter()
        return self
    def __exit__(self, *args):
        self.end = perf_counter()
        self.elapsed = self.end - self.start
        print(f"Elapsed time: {self.elapsed:.6f} seconds")


In [55]:
# Create a feasible LP problem and save to a
generate_feasible_lp(num_vars=6, num_ineq=4, num_eq=4, density=0.95, mps_filename="large_example.mps")

✅ LP written to: large_example.mps


/tmp/ipython-input-5-2499066835.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pulp.LpVariable(f"x{i}", lowBound=float(l[i]), upBound=float(u[i]))
/tmp/ipython-input-5-2499066835.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob += pulp.lpDot(G[i], x_vars) <= float(h[i]), f"ineq_{i}"
/tmp/ipython-input-5-2499066835.py:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob += pulp.lpDot(A[i], x_vars) == float(b[i]), f"eq_{i}"


In [56]:
import cplex

# Solve the LP using either primal simplex, dual simplex, or barrier (interior point).
# Only works for LP with no more than 1000 constraints and no more than 1000 variables
cpx = cplex.Cplex("large_example.mps")

# Times how long it takes to solve
with Timer():
    cpx.solve()

# Save the minimizer and minimal objective values for comparison
cpx_obj_val = cpx.solution.get_objective_value()
cpx_min = cpx.solution.get_values()
print("Objective value:", cpx_obj_val)
print("Minimizer: xᵀ =", cpx_min)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.01 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =           -32.367428
Elapsed time: 0.013115 seconds
Objective value: -28.333108832267087
Minimizer: xᵀ = [3.371809324017562, -5.615219725403, -10.261794339408421, -10.80378959065, 6.534395863185828, 9.73300777284794]


In [57]:
# Extract LP parameters from generated example
with Timer():
    c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")
    pdhg_obj_val, pdhg_min, iterations = pdhg(c, G, h, A, b, l, u, max_iter=1000000, tol=1e-8)
print("Objective Value:", pdhg_obj_val)
print("Iterations:", iterations)
print("Minimizer: xᵀ =",pdhg_min)

# The distance between the two minimizer solutions
# Should be small but won't be incredibly small since the vectors are high dimensional
#distance = np.linalg.norm(np.array(pdhg_min) - np.array(cpx_min))
#print("Distance:", distance)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
-33.36182673817835 -15.326473404286066 0
-28.43007752369969 -28.347757469902128 1000
-28.37302846154144 -28.33843298306913 2000
-28.3493746732931 -28.335033772597825 3000
-28.33968272549098 -28.33380061040593 4000
-28.335746701745762 -28.333355779371473 5000
-28.334160564696504 -28.33319632181417 6000
-28.333525746560458 -28.333139561135084 7000
-28.333273229197374 -28.333119516734747 8000
-28.333173341359128 -28.3331125029304 9000
-28.333134030538737 -28.333110075007912 10000
-28.333118633009644 -28.333109245348645 11000
-28.333112628733055 -28.333108966322676 12000
-28.333110297166282 -28.33310887436651 13000
-28.333109395389943 -28.333108844866153 14000
Elapsed time: 0.401561 seconds
Objective Value: -28.333108844866153
Iterations: 14001
Minimizer: xᵀ = [3.371809328851481, -5.615219725403, -10.261794346691847, -10.80378959065, 6.534395858545225, 9.7330077

In [58]:
# Extract LP parameters from generated example
with Timer():
  c, G, h, A, b, l, u = mps_to_standard_form("large_example.mps")
  pdhg_obj_val, pdhg_min, iterations = pdhg_precond(c, G, h, A, b, l, u, max_iter=1000000, tol=1e-8)
print("Objective Value:", pdhg_obj_val)
print("Iterations:", iterations)
print("Minimizer: xᵀ =",pdhg_min)

# The distance between the two minimizer solutions
# Should be small but won't be incredibly small since the vectors are high dimensional
#distance = np.linalg.norm(np.array(pdhg_min) - np.array(cpx_min))
#print("Distance:", distance)


Selected objective sense:  MINIMIZE
Selected objective  name:  OBJ
Selected RHS        name:  RHS
Selected bound      name:  BND
Duality gap: -20.474026, Primal obj: -15.323857, Iter: 0
Duality gap: -0.021258, Primal obj: -28.322230, Iter: 1000
Duality gap: -0.034707, Primal obj: -28.331762, Iter: 2000
Duality gap: -0.008414, Primal obj: -28.333674, Iter: 3000
Duality gap: -0.000033, Primal obj: -28.333396, Iter: 4000
Duality gap: -0.000739, Primal obj: -28.333144, Iter: 5000
Duality gap: -0.000216, Primal obj: -28.333094, Iter: 6000
Duality gap: -0.000016, Primal obj: -28.333101, Iter: 7000
Duality gap: -0.000024, Primal obj: -28.333108, Iter: 8000
Duality gap: -0.000006, Primal obj: -28.333109, Iter: 9000
Duality gap: -0.000000, Primal obj: -28.333109, Iter: 10000
Converged at iteration 10000
Elapsed time: 0.161055 seconds
Objective Value: -28.33310903185921
Iterations: 10001
Minimizer: xᵀ = [3.3718093935082307, -5.615219725403, -10.261794450448063, -10.80378959065, 6.53439578809931